In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
from keras.utils import np_utils
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional


from sklearn.model_selection import train_test_split

import sys
import os
import re
from collections import defaultdict
from nltk import ngrams



# fix random seed for reproducibility
np.random.seed(7)


# Put the hyperparameters at the top like this to make it easier to change and edit.

max_features = 20000
embedding_dim = 64
max_length = 60
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_portion = .8
num_classes = 3

Using TensorFlow backend.


In [2]:
cwd = os.getcwd()                      # Get the current working directory (cwd)
files = os.listdir(cwd)                # Get all the files in that directory

# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Print the type of the returned object
print("os.getcwd() returns an object of type: {0}".format(type(cwd)))



# Change the current working directory
# os.chdir('/tmp')

Current working directory: /home/asmelash/projects/mentoring/asfaw/deep-learning/code
os.getcwd() returns an object of type: <class 'str'>


In [3]:
#File path

filename = '../data/sources.tsv'

# File to read 

raw_text = open(filename, 'r', encoding='utf-8').read()

# Check all of the distinct characters in the file 
# This helps to remove unwanted characters(to clean the text)
raw_text  = re.sub(r"["":፣ ፡፡''፡“ ” ።?::፥፤]+\ *", " ",raw_text)

characters = sorted(list(set(raw_text)))
print(characters)


['\t', '\n', ' ', '!', '"', '#', '%', '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '©', '÷', 'ሀ', 'ሁ', 'ሂ', 'ሃ', 'ሄ', 'ህ', 'ሆ', 'ለ', 'ሉ', 'ሊ', 'ላ', 'ሌ', 'ል', 'ሎ', 'ሐ', 'ሑ', 'ሒ', 'ሓ', 'ሕ', 'ሖ', 'መ', 'ሙ', 'ሚ', 'ማ', 'ሜ', 'ም', 'ሞ', 'ሟ', 'ሥ', 'ረ', 'ሩ', 'ሪ', 'ራ', 'ሬ', 'ር', 'ሮ', 'ሰ', 'ሱ', 'ሲ', 'ሳ', 'ሴ', 'ስ', 'ሶ', 'ሸ', 'ሹ', 'ሺ', 'ሻ', 'ሼ', 'ሽ', 'ሾ', 'ቀ', 'ቁ', 'ቂ', 'ቃ', 'ቄ', 'ቅ', 'ቆ', 'ቈ', 'ቊ', 'ቋ', 'ቍ', 'ቐ', 'ቑ', 'ቒ', 'ቓ', 'ቕ', 'ቖ', 'ቘ', 'ቚ', 'ቛ', 'ቝ', 'በ', 'ቡ', 'ቢ', 'ባ', 'ቤ', 'ብ', 'ቦ', 'ቧ', 'ቨ', 'ቪ', 'ቫ', 'ቬ', 'ቭ', 'ተ', 'ቱ', 'ቲ', 'ታ', 'ቴ', 'ት', 'ቶ', 'ቸ', 'ቹ', 'ቺ', 'ቻ', 'ቼ', 'ች', 'ቾ', 'ኃ', 'ኅ', 'ነ', 'ኑ', 'ኒ', 'ና', 'ኔ', 'ን', 'ኖ', 'ኛ', 'ኝ', 'አ', 'ኡ', 'ኢ', 'ኣ', 'ኤ', 'እ', 'ኦ', 'ኧ', 'ከ', 'ኩ', 'ኪ', 'ካ', 'ኬ

In [4]:
def load_data(fin):
    '''accepts fin a string representing file name
    returns list of lines in file'''
    X = []
    y = []
    with open(fin, encoding="utf8") as f:
    
        labels = next(f).rstrip('\n').split('\t')
        for line in f:
            parts = line.rstrip('\n').split('\t')
            if len(set(parts)) != 3:
                continue
            for part, label in zip(parts, labels):

                X.append(part)
                y.append(label)
    return X,y

In [5]:
X,y = load_data('../../data/20210621-z_l_d-sents-all.tsv')

In [6]:
# Tokenization
tokenizer = Tokenizer(num_words = max_features, oov_token=oov_tok)
tokenizer.fit_on_texts(raw_text.split())
word_index = tokenizer.word_index
dict(list(word_index.items())[0:10])

{'<OOV>': 1,
 'd': 2,
 'z': 3,
 'l': 4,
 'ኣብ': 5,
 'እዩ': 6,
 'ካብ': 7,
 'ምስ': 8,
 'ዕርቂ': 9,
 'ሃም': 10}

In [7]:
len(word_index)

26068

In [8]:
X_tok = tokenizer.texts_to_sequences(X)

In [9]:
X[0].split()

['ወዲ',
 'ሰብ፣',
 'ካብ',
 'ፍጥረታት',
 'ዓለምና',
 'ዅሉ',
 'በዓል',
 'ብሩህ',
 'ኣእምሮ፣',
 'መስተውዓሊ፣',
 'ብልሂ፣',
 'ብምዃኑ',
 'ዘጋጥምዎ',
 'ፀገማት፣',
 'ፈተናታት',
 'ከባቢኡ',
 'ናብ',
 'ረብሕኡ',
 'ንምልዋጥ፣',
 'ብዙሓት',
 'መፍትሒ',
 'ምህዞታት፣',
 'ሜላታት',
 'ፈጢሩ፡፡']

In [10]:
X_tok[1]

[75,
 1,
 7,
 8278,
 526,
 602,
 162,
 2296,
 1,
 1,
 1,
 251,
 1,
 1,
 2724,
 720,
 31,
 4939,
 1,
 70,
 464,
 1,
 4940,
 1]

In [11]:
# Why is ሰብ፣' an OOV (Out of vocabulary)
# Preprocessing should be consitently applied in all steps: vocabulary generation, training, testing etc.

X_preprocessed = [re.sub(r"["":፣ ፡፡''፡“ ” ።?::፥፤]+\ *", " ",x) for x in X]
X_tok = tokenizer.texts_to_sequences(X_preprocessed)

In [12]:
X_preprocessed[0].split()

['ወዲ',
 'ሰብ',
 'ካብ',
 'ፍጥረታት',
 'ዓለምና',
 'ዅሉ',
 'በዓል',
 'ብሩህ',
 'ኣእምሮ',
 'መስተውዓሊ',
 'ብልሂ',
 'ብምዃኑ',
 'ዘጋጥምዎ',
 'ፀገማት',
 'ፈተናታት',
 'ከባቢኡ',
 'ናብ',
 'ረብሕኡ',
 'ንምልዋጥ',
 'ብዙሓት',
 'መፍትሒ',
 'ምህዞታት',
 'ሜላታት',
 'ፈጢሩ']

In [13]:
X_tok[0]

[75,
 12,
 7,
 8278,
 526,
 602,
 162,
 2296,
 819,
 4937,
 4938,
 251,
 13608,
 3430,
 2724,
 720,
 29,
 4939,
 8279,
 70,
 464,
 1908,
 4940,
 1527]

In [14]:
max([len(X_tok[i]) for i in range(len(X_tok))])

54

In [15]:
for idx, i in enumerate(range(len(X_tok))):
    if len(X_tok[i]) == 54:
        print(idx)

12
13


In [16]:
X[12]

'ህዝብታት ዓለም ዝሓለፉዎ ታሪክ ጐንፅታት ሕድሕድ፥ ኣብ ኣህጉርና፣ ኣብ ሃገርና ኢትዮጵያ ኰነ ኣብ ትግራይ ብዓሌት፣ ብዘርኢ፣ ብከባቢ፣ ብብሄር፣ ሓው ምስ ሓው ባእሲ፣ ብነብሲ ምድልላይ፣ ኣተሓሳስባ ድሕረት ጐዳእቲ ባህላዊ ፍፃመታት ካብ ዝበሃሉ ሰብ ንሰብ ብምስዓሩ፣ ብምጕድኡ ወይከዓ ብምቕታሉ፣ መላሲ ደም፣ ፈዳይ ሕነ እናተብሃለ ከም ፍሉይ መለክዒ ጅግንነት ተገይሩ እናተቘፀረሉ ዝነበረ ዘመናት፥ ተዘክሮ ቀረባ እዋን እዩ፡፡'

In [17]:
X[13]

'ህዝብታት ዓለም ልሓለፉዎ ታሪክ ጐንፅታት ሕድሕድ፥ ኣብ ኣህጉርና፣ ኣብ ሃገርና ኢትዮጵያ ኰነ ኣብ ትግራይ ብዓሌት፣ ብዘርኢ፣ ብከባቢ፣ ብብሄር፣ ሓው ምስ ሓው ባእሲ፣ ብነብሲ ምድልላይ፣ ኣተሓሳስባ ድሕረት ጐዳእቲ ባህላዊ ፍፃመታት ካብ ልበሃሉ ሰብ ልሰብ ብምስዓሩ፣ ብምጕድኡ ወይ ብምቕታሉ፣ መላሲ ደም፣ ፈዳይ ሕነ ላተብሃለ ሃም ፍሉይ መለክዒ ጅግንነት ተገይሩ ላተቘፀረሉ ልነበረ ዘበናት፥ ተዘክሮ ቀረባ እዋን እዩ፡፡'

In [18]:
X_tok_padded = pad_sequences(X_tok, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [19]:
X_tok_padded[0]

array([   75,    12,     7,  8278,   526,   602,   162,  2296,   819,
        4937,  4938,   251, 13608,  3430,  2724,   720,    29,  4939,
        8279,    70,   464,  1908,  4940,  1527,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0], dtype=int32)

In [130]:
#our labels are in Z, L, D but we want a three dimensional vector corresponding to each class
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
y_ = lb.fit_transform(y)

In [21]:
print(y[:5])

['Z', 'L', 'D', 'Z', 'L']


In [22]:
print(y_[:5])

[[0 0 1]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]]


In [125]:
def load_feature(fin):
    '''accepts fin a string representing file name of a feature file e.g., char ngrams
    returns list of lines in file'''
    X = []
    y = []
    with open(fin, encoding="utf8") as f:
        for line in f:
            parts = line.rstrip('\n').split('\t')
            if len(set(parts)) != 2:
                continue
            label, feature = parts
            
            try:
                X.append(np.array([int(s) for s in feature.split()]))
                y.append(label)
            except:
                #print(parts)
                continue
    return X,y

In [141]:
# Character ngrams
X,y = load_feature('../data/ngram_1024.tsv')

lb = preprocessing.LabelBinarizer()
y = lb.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [140]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tok_padded, y_, test_size=0.2, random_state=42)

print(len(X))
print(len(y))

print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

ValueError: Found input variables with inconsistent numbers of samples: [7974, 8315]

In [24]:
X[0]

'ወዲ ሰብ፣ ካብ ፍጥረታት ዓለምና ዅሉ በዓል ብሩህ ኣእምሮ፣ መስተውዓሊ፣ ብልሂ፣ ብምዃኑ ዘጋጥምዎ ፀገማት፣ ፈተናታት ከባቢኡ ናብ ረብሕኡ ንምልዋጥ፣ ብዙሓት መፍትሒ ምህዞታት፣ ሜላታት ፈጢሩ፡፡'

In [25]:
X_train[0]

array([15966,  2493,   451, 15967,    12, 15968,  2610, 15969,    27,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0], dtype=int32)

In [138]:
# create a LSTM model
model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(max_features, embedding_dim),
    tf.keras.layers.LSTM(embedding_dim),
#    tf.kerafs.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # use ReLU in place of tanh function since they are very good alternatives of each other.
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    # Add a Dense layer with 3 units and softmax activation.
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, None, 64)          1280000   
_________________________________________________________________
lstm_9 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_16 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_17 (Dense)             (None, 3)                 195       
Total params: 1,317,379
Trainable params: 1,317,379
Non-trainable params: 0
_________________________________________________________________


In [140]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 10
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test), verbose=2)

Train on 6379 samples, validate on 1595 samples
Epoch 1/10
6379/6379 - 16s - loss: 1.0989 - accuracy: 0.3237 - val_loss: 1.0987 - val_accuracy: 0.3223
Epoch 2/10
6379/6379 - 10s - loss: 1.0991 - accuracy: 0.3378 - val_loss: 1.0993 - val_accuracy: 0.3423
Epoch 3/10
6379/6379 - 10s - loss: 1.0990 - accuracy: 0.3292 - val_loss: 1.0986 - val_accuracy: 0.3354
Epoch 4/10
6379/6379 - 11s - loss: 1.0989 - accuracy: 0.3281 - val_loss: 1.0987 - val_accuracy: 0.3223
Epoch 5/10
6379/6379 - 11s - loss: 1.0987 - accuracy: 0.3361 - val_loss: 1.0987 - val_accuracy: 0.3223
Epoch 6/10
6379/6379 - 11s - loss: 1.0987 - accuracy: 0.3256 - val_loss: 1.0987 - val_accuracy: 0.3223
Epoch 7/10
6379/6379 - 12s - loss: 1.0987 - accuracy: 0.3334 - val_loss: 1.0987 - val_accuracy: 0.3223
Epoch 8/10
6379/6379 - 9s - loss: 1.0987 - accuracy: 0.3286 - val_loss: 1.0987 - val_accuracy: 0.3223
Epoch 9/10
6379/6379 - 11s - loss: 1.0987 - accuracy: 0.3316 - val_loss: 1.0988 - val_accuracy: 0.3223
Epoch 10/10
6379/6379 - 10

In [142]:
# create a Bi-LSTM model
model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(max_features, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
#    tf.kerafs.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # use ReLU in place of tanh function since they are very good alternatives of each other.
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    # Add a Dense layer with 3 units and softmax activation.
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, None, 64)          1280000   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_18 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_19 (Dense)             (None, 3)                 195       
Total params: 1,354,499
Trainable params: 1,354,499
Non-trainable params: 0
_________________________________________________________________


In [143]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 10
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test), verbose=2)

Train on 6379 samples, validate on 1595 samples
Epoch 1/10
6379/6379 - 18s - loss: 0.8771 - accuracy: 0.5408 - val_loss: 0.6696 - val_accuracy: 0.6583
Epoch 2/10
6379/6379 - 11s - loss: 0.4324 - accuracy: 0.8141 - val_loss: 0.6191 - val_accuracy: 0.7003
Epoch 3/10
6379/6379 - 11s - loss: 0.2016 - accuracy: 0.9215 - val_loss: 0.7462 - val_accuracy: 0.7185
Epoch 4/10
6379/6379 - 11s - loss: 0.1183 - accuracy: 0.9549 - val_loss: 0.8593 - val_accuracy: 0.7223
Epoch 5/10
6379/6379 - 11s - loss: 0.0788 - accuracy: 0.9713 - val_loss: 0.9009 - val_accuracy: 0.7097
Epoch 6/10
6379/6379 - 11s - loss: 0.0669 - accuracy: 0.9785 - val_loss: 0.9307 - val_accuracy: 0.7166
Epoch 7/10
6379/6379 - 11s - loss: 0.0507 - accuracy: 0.9828 - val_loss: 1.0516 - val_accuracy: 0.7335
Epoch 8/10
6379/6379 - 11s - loss: 0.0445 - accuracy: 0.9828 - val_loss: 1.0660 - val_accuracy: 0.7266
Epoch 9/10
6379/6379 - 11s - loss: 0.0381 - accuracy: 0.9856 - val_loss: 1.1306 - val_accuracy: 0.7379
Epoch 10/10
6379/6379 - 1

In [26]:
# referece keras.io
from tensorflow.keras import layers

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [142]:
def cnn_model(drop_out, filter_one, filter_two, last_dense, padding):
    # create a CNN model
    model = tf.keras.Sequential([
        # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
        tf.keras.layers.Embedding(max_features, embedding_dim),

        #TODO: suspect this is a good idea to add
        #tf.keras.layers.Dense(embedding_dim, activation='relu')
        tf.keras.layers.Dropout(drop_out),
        tf.keras.layers.Conv1D(filters=filter_one, kernel_size=6, padding=padding, activation='relu'),
        #tf.keras.layers.MaxPool1D(pool_size=4),

        #tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Conv1D(filters=filter_two, kernel_size=5, padding=padding, activation='relu'),
        tf.keras.layers.GlobalMaxPool1D(),
        tf.keras.layers.Dropout(drop_out),

        #tf.keras.layers.LSTM(100),
        tf.keras.layers.Dense(last_dense, activation='relu'),

        # Add a Dense layer with 3 units and softmax activation.
        # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.summary()
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    num_epochs = 1
    history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test), verbose=2)
    
    return history.history

In [144]:
max_features = 1024
embedding_dim = 64
cnn_model(drop_out=0.5, filter_one=32, filter_two=32, last_dense=128, padding='same')

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, None, 64)          65536     
_________________________________________________________________
dropout_39 (Dropout)         (None, None, 64)          0         
_________________________________________________________________
conv1d_40 (Conv1D)           (None, None, 32)          12320     
_________________________________________________________________
conv1d_41 (Conv1D)           (None, None, 32)          5152      
_________________________________________________________________
global_max_pooling1d_17 (Glo (None, 32)                0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 128)             

ValueError: Input arrays should have the same number of samples as target arrays. Found 1024 input samples and 6652 target samples.

In [97]:
def run_experiments():
    # grid search
    drop_outs = (0.5, 0.7, 0.8)
    filters_one = (32, 64, 128)
    filters_two = (32, 64, 128)
    last_dense_layers = (32, 64, 128)
    paddings = ('same', 'valid')
    
    results = []
    for drop_out in drop_outs:
        for filter_one in filters_one[:-1]:
            for filter_two in filters_two:
                for last_dense in last_dense_layers:
                    for padding in paddings:
                        res = cnn_model(drop_out, filter_one, filter_two, last_dense, padding)
                        #res = 'running ...'
                        results.append({'drop_out': drop_out,
                                   'filter_one': filter_one,
                                   'filter_two': filter_two,
                                    'last_dense': last_dense,
                                    'padding': padding,
                                    'history': res})
    return results

In [98]:
r = run_experiments()

In [91]:
l = [1,2,3]